In [1]:
import ee
import geemap
from datetime import date, timedelta, datetime
from omegaconf import DictConfig, OmegaConf

In [2]:
ee.Authenticate()

Enter verification code:  4/1AeaYSHCIXn5CX0gV_19NOjtqcBeJrM4-181N1YUKo0kpF-JwtlEzHGJX2C0



Successfully saved authorization token.


# Extracting Scene ID

Acquiring WRS: https://landsat.usgs.gov/landsat_acq#convertPathRow (you can obtain scene_id here)</br> 
Landsat Naming Conventions: https://gisgeography.com/landsat-file-naming-convention/ </br>
LAndsat 7/8/9 Acquisition Calendar: https://landsat.usgs.gov/landsat_acq#convertPathRow </br>

# Extracting Region of Interest (ROI) for cities

In [3]:
def acquire_reference_date(start_date, scene_id):
    """
    Find the first date after which a LANDSAT 8 image is valid for a given scene id
    :param start_date:
    :param scene_id:
    :return:
    """
    reference_date = None
    cur_date = datetime.strptime(start_date, '%Y%m%d')
    while reference_date is None:
        cur_date_str = datetime.strftime(cur_date, '%Y%m%d')
        try:
            img = ee.Image(f'LANDSAT/LC08/C02/T1_TOA/LC08_{scene_id}_{cur_date_str}')
            img.getInfo()
        except ee.EEException as e:  # image does not exist
            cur_date = cur_date + timedelta(days=1)
            continue
        # image exists, in the case of no exception
        reference_date = cur_date_str
        return reference_date

In [4]:
ee.Initialize()
sr_vis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3,} # for LANDSAT8 surface reflectance RGB
print('Please provide a scene_id for corresponding to a LANDSAT8 image')
scene_id = input()
if len(scene_id) == 5:
    scene_id = '0' + scene_id
    print('zero padded to ', scene_id)

Please provide a scene_id for corresponding to a LANDSAT8 image


 023032


In [5]:
ref_date = acquire_reference_date(start_date='20180701', scene_id=scene_id)
Map = geemap.Map(center=(39, -98), zoom=4)
img = ee.Image(f'LANDSAT/LC08/C02/T1_TOA/LC08_{scene_id}_{ref_date}').select(['B4', 'B3', 'B2'])
Map.addLayer(img, {'min': 0.0, 'max': 0.4})
print('Please draw a rectangular bounding box within the image')
Map

Please draw a rectangular bounding box within the image


Map(center=[39, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [27]:
meta_ = Map.user_rois.getInfo()
coord = meta_['features'][0]['geometry']['coordinates']
print(coord)

[[[-81.003595, 35.034301], [-80.609429, 35.034301], [-80.609429, 35.373182], [-81.003595, 35.373182], [-81.003595, 35.034301]]]


In [11]:
# img.getInfo()

# Extracting ROI for SURFRAD

In [ ]:
config = OmegaConf.load('../config/surfrad.yaml')
for station in config.stations:
    print(station)

BND
TBL
DRA
FPK
GWN
PSU
SXF
SGP


#### Now repeat for each station, place a pin, and draw a bounding box

In [24]:
station = 'SXF'
lon, lat = config['stations'][station]['Longitude'], config['stations'][station]['Latitude']
scene_id = config['stations'][station]['scene_id']
ref_date = acquire_reference_date(start_date='20180701', scene_id=scene_id)
Map = geemap.Map()
Map.set_center(lon, lat, 15)
img = ee.Image(f'LANDSAT/LC08/C02/T1_TOA/LC08_{scene_id}_{ref_date}').select(['B4', 'B3', 'B2'])
Map.addLayer(img, {'min': 0.0, 'max': 0.4})
print('Please draw a rectangular bounding box within the image')
Map

Please draw a rectangular bounding box within the image


Map(center=[43.73403, -96.62328], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [23]:
meta_ = Map.user_rois.getInfo()
coord = meta_['features'][1]['geometry']['coordinates']
print(coord)

[[[-90.631714, 34.052339], [-89.689636, 34.052339], [-89.689636, 34.790616], [-90.631714, 34.790616], [-90.631714, 34.052339]]]


# Additionals

In [13]:
import pandas as pd

In [19]:
df = pd.read_csv('../data/us_cities.csv')
df

,2021 rank,City,State[c],Scene ID,Notes
0,1,New York,New York,13032,NaN
1,2,Los Angeles,California,41036,cut off Long Beach
2,3,Chicago,Illinois,23031,NaN
3,4,Houston,Texas,25039,NaN
4,5,Phoenix,Arizona,37037,NaN
5,6,Philadelphia,Pennsylvania,14032,NaN
6,7,San Antonio,Texas,27040,NaN
7,8,San Diego,California,40037,NaN
8,9,Dallas,Texas,27037,image includes Fort Worth
9,10,San Jose,California,44034,same as San Francisco
